#  Introduction to LangGraph for Multi-Agent Systems

LangGraph is a framework that allows you to build stateful, multi-step, decision-based workflows using Large Language Models (LLMs). It's like drawing a flowchart — where each step (called a "node") does something useful, and the system decides what to do next based on data.

In this notebook, you'll see how to build a **simple multi-agent finance system** using LangGraph. It uses both **normal flow** and **conditional logic** to demonstrate how LangGraph helps you create smart agent workflows.

---

## 🔁 System Overview

### Use Case: Personal Finance Advisor
We want to build an AI system that:
1. Takes in user investment preferences.
2. Analyzes the user's risk tolerance.
3. Directs them to the right advisor:
   - 📉 Low risk → Bank Advisor
   - 📈 High risk → Stock Advisor

### LangGraph Elements Used:
| Element              | Description                                        |
|---------------------|----------------------------------------------------|
| **State**           | A dictionary holding data that flows between steps |
| **Nodes**           | Functions that process and update the state        |
| **Edges**           | Define the flow from one node to another           |
| **Conditional Edges** | Allow branching logic (if/else style decisions)    |
| **StateGraph**      | The LangGraph object that wires it all together    |
| **END**             | Marks the end of the flow                          |

---


# LangGraph Multi-Agent Finance System with AI Agent


In [ ]:
# --- 1. Install Required Packages ---
!pip install langgraph langchain openai langchain_community -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.9 MB/s eta 0:00:00


In [ ]:
# --- 2. Imports ---
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage
import os

In [ ]:
# Set your OpenAI API key (you can also set via environment variable)
os.environ["OPENAI_API_KEY"] = "sk-proj-SCddVD3ZWsbBhv5IScfoT3BlbkFJAJdyR6CDtI7PNCp7Uv0l"  # Replace with your actual key

In [ ]:
# --- 3. Define the State ---
# The shared data (state) that flows through the graph
class FinanceState(TypedDict):
    amount: float             # How much money the user wants to invest
    risk_level: str           # "low" or "high"
    recommendation: str       # Final recommendation from an advisor

In [ ]:
# --- 4. Define the Nodes ---

# Node 1: Intake Agent
# Collects the user's investment amount and risk preference

def intake_agent(state: FinanceState) -> FinanceState:
    print("\n📋 Intake Agent: Collecting user info...")
    try:
        state["amount"] = float(input("Enter the amount you want to invest (INR): "))
        risk = input("Enter your risk level (low/high): ").strip().lower()
        state["risk_level"] = risk if risk in ["low", "high"] else "low"
    except Exception as e:
        print("Invalid input. Using default values.")
        state["amount"] = 10000
        state["risk_level"] = "low"
    return state

# Node 2: Risk Analyzer (Conditional)
# Chooses which advisor to send the user to based on risk_level
def risk_analyzer(state: FinanceState):
    print(f"🧠 Risk Analyzer: Evaluating risk ({state['risk_level']})...")
    # Update state with the next node to visit
    state["next_node"] = "bank_advisor" if state["risk_level"] == "low" else "ai_advisor"
    return state

# Node 3: Bank Advisor (for low risk)
def bank_advisor(state: FinanceState) -> FinanceState:
    print("🏦 Bank Advisor: Recommending fixed deposits.")
    state["recommendation"] = "Invest in Fixed Deposits or Bonds"
    return state

# Node 4: AI-Powered Stock Advisor using GPT
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

def ai_stock_advisor(state: FinanceState) -> FinanceState:
    print("🤖 AI Stock Advisor: Generating investment advice with GPT...")
    user_prompt = f"The user has {state['amount']} INR and a high risk appetite. Suggest a smart investment strategy in Indian stocks."
    response = llm([SystemMessage(content=user_prompt)])
    state["recommendation"] = response.content
    return state

In [ ]:
# --- 5. Build the Graph ---

# Create the graph and register each node
builder = StateGraph(FinanceState)

builder.add_node("intake", intake_agent)
builder.add_node("risk_analyzer", risk_analyzer)
builder.add_node("bank_advisor", bank_advisor)
builder.add_node("ai_advisor", ai_stock_advisor)

# Set the first node to run
builder.set_entry_point("intake")

# Define normal edge: intake → risk_analyzer
builder.add_edge("intake", "risk_analyzer")

# Define conditional edge: risk_analyzer → bank_advisor or ai_advisor
# The path_map argument should map conditions to node names
# Changed from string to lambda function to extract risk_level from state
builder.add_conditional_edges("risk_analyzer", path=lambda state: state["next_node"], path_map={
    "bank_advisor": "bank_advisor",
    "ai_advisor": "ai_advisor"
})

# End the graph after advice is given
builder.add_edge("bank_advisor", END)
builder.add_edge("ai_advisor", END)

# Compile the graph so it's ready to run
graph = builder.compile()

In [ ]:
# --- 6. Run the Graph ---
# Create an initial state to start the flow
initial_state = {
    "amount": 0,
    "risk_level": "",
    "recommendation": ""
}

# Invoke the graph (start execution)
result = graph.invoke(initial_state)

# --- 7. Final Output ---
print("\n✅ Final Recommendation:")
print(result["recommendation"])



📋 Intake Agent: Collecting user info...
Enter the amount you want to invest (INR): 10000
Enter your risk level (low/high): low
🧠 Risk Analyzer: Evaluating risk (low)...
🏦 Bank Advisor: Recommending fixed deposits.

✅ Final Recommendation:
Invest in Fixed Deposits or Bonds
